# FastAPI Requests

This notebook demonstrates, how requests can be submitted to the running FastAPI.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import requests

In [3]:
with open('../html_body.txt', 'r', encoding='utf-8') as file:
    raw_html = file.read()

Check if API is up and running:

In [4]:
requests.get('http://127.0.0.1:8000/health').json()

'Comment API is healthy'

Submit the actual analysis request:

In [5]:
response = requests.post(
    url='http://127.0.0.1:8000/analyze-comments',
    json={
        "raw_html": raw_html,
        "params": {
            "analyze_donations": True,
            "target_currency": "USD",
            "dislike_polarity_cutoff": 0.3
        }
    }
)

In [6]:
response.status_code

200

If the status code is 200, we can check the metrics of the analysis below:

In [8]:
print(json.dumps(response.json(), indent=3))

{
   "comments": {
      "raw_comments__count": 225,
      "translated_comments__count": 221,
      "sentiment_comments__vader__median": 0.6369,
      "sentiment_comments__textblob__median": 0.2
   },
   "donations": {
      "donations__count": 80,
      "donations__sum": 1826.49,
      "donations__currency": "USD"
   },
   "dislikes": {
      "highly_positive_comments__count": 142,
      "highly_negative_comments__count": 3,
      "likes__count": 314865,
      "dislikes__estimate": 6652
   }
}
